In [1]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
import numpy as np
import matplotlib.pyplot as plt
import datetime,time
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from scipy import signal
import statistics

In [2]:
# Mean absolute error (MAE)

def MAE(Y, Y_chap):
    if len(Y) == len(Y_chap):
        mae = np.sum(np.absolute(Y_chap - Y)) / len(Y)
    else:
        print('Probleme de dimension entre Y et Y_chap')
        print('len(Y) :', len(Y))
        print('len(Y_chap) :', len(Y_chap))
    
    return mae


# Root Mean squared error (MRSE)

def MRSE(Y, Y_chap):
    if len(Y) == len(Y_chap):
        mrse = np.sqrt(((Y_chap - Y)**2).mean()) 
    else:
        print('Probleme de dimension entre Y et Y_chap')
        print('len(Y) :', len(Y))
        print('len(Y_chap) :', len(Y_chap))
    
    return mrse

## Turbine 1

In [3]:
Turb1 = pd.read_csv('./FRSSO18.csv',sep=';',decimal=',',parse_dates=['DateTime'],dayfirst=True)
Turb1

,DateTime,ActivePower,ProductionForecast_D,ProductionForecast_M
0,2020-03-05 00:00:00,1614.345422,948.189357,745.2
1,2020-03-05 01:00:00,1478.355419,967.269962,711.8
2,2020-03-05 02:00:00,1690.147069,948.830677,645.0
3,2020-03-05 03:00:00,1516.965994,1102.870568,645.0
4,2020-03-05 04:00:00,936.617865,962.811666,568.8
...,...,...,...,...
11673,2021-10-16 14:00:00,0.423586,0.000000,0.0
11674,2021-10-16 16:00:00,33.743984,0.000000,0.0
11675,2021-10-16 17:00:00,66.180937,29.035056,0.0
11676,2021-10-16 22:00:00,3.609505,0.000000,4.0


In [4]:
Turb1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11678 entries, 0 to 11677
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DateTime              11678 non-null  datetime64[ns]
 1   ActivePower           11678 non-null  float64       
 2   ProductionForecast_D  11678 non-null  float64       
 3   ProductionForecast_M  11678 non-null  float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 365.1 KB


In [5]:
# Si on veut un an de train et le reste de test, quel est l'indice de split dans le dataset? 
Turb1.loc[Turb1['DateTime']== '2021-03-05 00:00:00'].index.values

array([7276])

In [6]:
# On récupère la liste des indices des moments de prédiction (chaque jour à 12h)

indicedf = pd.DataFrame(Turb1['DateTime'])
liste_p1 = Turb1[indicedf.loc[:,'DateTime'].apply(lambda x: x.strftime('%H'))=='12'].index.values
liste_p1                           

array([   12,    36,    60,    84,   108,   132,   156,   179,   202,
         226,   250,   284,   306,   330,   354,   378,   400,   424,
         445,   468,   488,   512,   536,   559,   581,   605,   642,
         666,   690,   713,   729,   752,   782,   795,   819,   843,
         867,   882,   902,   922,   946,   969,   993,  1015,  1039,
        1076,  1098,  1122,  1145,  1167,  1190,  1208,  1229,  1253,
        1277,  1296,  1312,  1347,  1370,  1394,  1441,  1465,  1489,
        1503,  1518,  1541,  1569,  1589,  1613,  1634,  1658,  1680,
        1703,  1727,  1775,  1782,  1807,  1820,  1871,  1894,  1917,
        1941,  1963,  1986,  2008,  2032,  2056,  2086,  2138,  2158,
        2177,  2207,  2228,  2249,  2269,  2293,  2313,  2337,  2358,
        2381,  2420,  2442,  2466,  2490,  2514,  2538,  2555,  2595,
        2619,  2647,  2669,  2691,  2715,  2757,  2776,  2800,  2824,
        2841,  2891,  2909,  2931,  2953,  2971,  2990,  3014,  3044,
        3073,  3085,

In [7]:
Turb1[['ProductionForecast_M','ProductionForecast_D']][liste_p1[0]:liste_p1[2]]

,ProductionForecast_M,ProductionForecast_D
12,2028.0,2057.518240
13,2032.0,2057.500489
14,2044.0,2057.209959
15,2050.0,2047.839547
16,2042.0,2044.552384
17,2008.0,1984.293343
18,1959.0,2057.212053
19,1918.0,2057.508598
20,2000.0,2057.403097
21,2032.0,2057.494224


In [17]:
smae_1 = 0
smrse_1 = 0
l1 = 0
y_pred1 = []

for i in range(0, len(liste_p1) - 62):
    x_train1_i = np.array(Turb1[['ProductionForecast_M','ProductionForecast_D']][liste_p1[i]:liste_p1[i+60]])
    x_test1_i = np.array(Turb1[['ProductionForecast_M','ProductionForecast_D']][liste_p1[i+60]:liste_p1[i+62]])
    y_train1_i = np.array(Turb1[['ActivePower']][liste_p1[i]:liste_p1[i+60]])
    y_test1_i = np.array(Turb1[['ActivePower']][liste_p1[i+60]:liste_p1[i+62]])
    reg1_i = LinearRegression().fit(x_train1_i, y_train1_i)
    y_pred1_i = reg1_i.predict(x_test1_i)
    smae_1 += np.sum(np.absolute(y_test1_i - y_pred1_i))
    smrse_1 += np.sum((y_test1_i - y_pred1_i)**2)
    l1 += len(y_test1_i)
    y_pred1.append(y_pred1_i)
    
    #mae1_i = MAE(y_test1_i, y_pred1_i)
    #mrse1_i = MRSE(y_test1_i, y_pred1_i)
    #mae_1.append(mae1_i) 
    #mrse_1.append(mrse1_i)

In [18]:
mae_turb1 = smae_1/l1
mrse_turb1 = np.sqrt(smrse_1/l1)
#mae_turb1 = statistics.mean(mae_1)
#mrse_turb1 = statistics.mean(mrse_1)
print('MAE_turb1 :', mae_turb1)
print('MRSE_turb1 :', mrse_turb1)

MAE_turb1 : 217.41411917748542
MRSE_turb1 : 308.6588454490928


In [43]:
print(l1) # plus grande longueur que le dataset car les prédictions se recouvrent
print(len(y_pred1)) # on retrouve le nombre de midis du dataset

22720
466


In [19]:
# Valeurs de référence des experts 
mae1_D = MAE(np.array((Turb1[['ProductionForecast_D']][1394:])), np.array(Turb1[['ActivePower']][1394:]))
mae1_M = MAE(np.array((Turb1[['ProductionForecast_M']][1394:])), np.array(Turb1[['ActivePower']][1394:]))
mrse1_D = MRSE(np.array((Turb1[['ProductionForecast_D']][1394:])), np.array(Turb1[['ActivePower']][1394:]))
mrse1_M = MRSE(np.array((Turb1[['ProductionForecast_M']][1394:])), np.array(Turb1[['ActivePower']][1394:]))
print('MAE_D :', mae1_D)
print('MRSE_D :', mrse1_D)
print('MAE_M :', mae1_M)
print('MRSE_M :', mrse1_M)

MAE_D : 229.1306128468052
MRSE_D : 349.0180130285433
MAE_M : 217.0059174462975
MRSE_M : 327.70820880504647


## Turbine 2

In [20]:
Turb2 = pd.read_csv('./FRMHN11.csv',sep=';',decimal=',',parse_dates=['DateTime'],dayfirst=True)
Turb2

,DateTime,ActivePower,ProductionForecast_D,ProductionForecast_M
0,2020-03-05 00:00:00,1697.202216,1750.206339,1338.75
1,2020-03-05 01:00:00,1708.172571,1859.345119,1143.75
2,2020-03-05 02:00:00,1244.708609,1727.167637,1046.25
3,2020-03-05 03:00:00,1261.804036,1645.022462,1046.25
4,2020-03-05 04:00:00,1281.095993,1655.782922,997.50
...,...,...,...,...
11757,2021-10-20 00:00:00,3299.856095,2720.159697,2820.00
11758,2021-10-20 01:00:00,2387.907817,2660.177096,2820.00
11759,2021-10-20 02:00:00,2016.272911,2568.792496,2775.00
11760,2021-10-20 03:00:00,2181.557309,2739.319746,2775.00


In [21]:
Turb2.loc[Turb2['DateTime']== '2021-03-05 00:00:00'].index.values

array([7221])

In [22]:
# On récupère la liste des indices des moments de prédiction (chaque jour à 12h)

indicedf2 = pd.DataFrame(Turb1['DateTime'])
liste_p2 = Turb1[indicedf2.loc[:,'DateTime'].apply(lambda x: x.strftime('%H'))=='12'].index.values
liste_p2    

array([   12,    36,    60,    84,   108,   132,   156,   179,   202,
         226,   250,   284,   306,   330,   354,   378,   400,   424,
         445,   468,   488,   512,   536,   559,   581,   605,   642,
         666,   690,   713,   729,   752,   782,   795,   819,   843,
         867,   882,   902,   922,   946,   969,   993,  1015,  1039,
        1076,  1098,  1122,  1145,  1167,  1190,  1208,  1229,  1253,
        1277,  1296,  1312,  1347,  1370,  1394,  1441,  1465,  1489,
        1503,  1518,  1541,  1569,  1589,  1613,  1634,  1658,  1680,
        1703,  1727,  1775,  1782,  1807,  1820,  1871,  1894,  1917,
        1941,  1963,  1986,  2008,  2032,  2056,  2086,  2138,  2158,
        2177,  2207,  2228,  2249,  2269,  2293,  2313,  2337,  2358,
        2381,  2420,  2442,  2466,  2490,  2514,  2538,  2555,  2595,
        2619,  2647,  2669,  2691,  2715,  2757,  2776,  2800,  2824,
        2841,  2891,  2909,  2931,  2953,  2971,  2990,  3014,  3044,
        3073,  3085,

In [23]:
smae_2 = 0
smrse_2 = 0
l2 = 0

for i in range(0, len(liste_p1) - 62):
    x_train2_i = np.array(Turb2[['ProductionForecast_M','ProductionForecast_D']][liste_p2[i]:liste_p2[i+60]])
    x_test2_i = np.array(Turb2[['ProductionForecast_M','ProductionForecast_D']][liste_p2[i+60]:liste_p2[i+62]])
    y_train2_i = np.array(Turb2[['ActivePower']][liste_p2[i]:liste_p2[i+60]])
    y_test2_i = np.array(Turb2[['ActivePower']][liste_p2[i+60]:liste_p2[i+62]])
    reg2_i = LinearRegression().fit(x_train2_i, y_train2_i)
    y_pred2_i = reg2_i.predict(x_test2_i)
    smae_2 += np.sum(np.absolute(y_test2_i - y_pred2_i))
    smrse_2 += np.sum((y_test2_i - y_pred2_i)**2)
    l2 += len(y_test2_i)

    #mae2_i = MAE(y_test2_i, y_pred2_i)
    #mrse2_i = MRSE(y_test2_i, y_pred2_i)
    #mae_2.append(mae2_i) 
    #mrse_2.append(mrse2_i)

In [24]:
mae_turb2 = smae_2/l2
mrse_turb2 = np.sqrt(smrse_2/l2)
#mae_turb2 = statistics.mean(mae_2)
#mrse_turb2 = statistics.mean(mrse_2)
print('MAE_turb2 :', mae_turb2)
print('MRSE_turb2 :', mrse_turb2)

MAE_turb2 : 326.3025119450661
MRSE_turb2 : 469.43941893793794


In [27]:
# Valeurs de référence des experts 
mae2_D = MAE(np.array((Turb2[['ProductionForecast_D']][1394:])), np.array(Turb2[['ActivePower']][1394:]))
mae2_M = MAE(np.array((Turb2[['ProductionForecast_M']][1394:])), np.array(Turb2[['ActivePower']][1394:]))
mrse2_D = MRSE(np.array((Turb2[['ProductionForecast_D']][1394:])), np.array(Turb2[['ActivePower']][1394:]))
mrse2_M = MRSE(np.array((Turb2[['ProductionForecast_M']][1394:])), np.array(Turb2[['ActivePower']][1394:]))
print('MAE_D :', mae2_D)
print('MRSE_D :', mrse2_D)
print('MAE_M :', mae2_M)
print('MRSE_M :', mrse2_M)

MAE_D : 362.7106966940372
MRSE_D : 544.328521370032
MAE_M : 321.93381215981174
MRSE_M : 490.493708397833
